In [4]:
# --- Local OpenAI-compatible client (LM Studio / Ollama) ---
# Endpoint: http://127.0.0.1:1234  (your server)
# Model:    qwen/qwen2.5-vl-7b

%pip install -q openai

import os
from openai import OpenAI

BASE_URL = "http://127.0.0.1:1234/v1"      # <— your server
API_KEY  = "sk-local"                      # <— dummy key, any string works
MODEL = "qwen/qwen2.5-vl-7b"               # <— from your models list

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

def local_chat(prompt: str, *, system: str | None = None,
               temperature: float = 0.2, max_tokens: int = 512,
               model: str = "qwen/qwen2.5-vl-7b") -> str:
    msgs = []
    if system:
        msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": prompt})
    resp = client.chat.completions.create(
        model=model,
        messages=msgs,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    raw = resp.choices[0].message.content.strip()
    return clean_thinking_output(raw)


def clean_thinking_output(text: str) -> str:
    # If the model outputs <think>…</think>, strip that part
    if "</think>" in text:
        return text.split("</think>")[-1].strip()
    if "<think>" in text:
        return text.split("<think>")[-1].strip()
    return text.strip()    
    

# quick smoketest:
print(local_chat("Say exactly: setup successful."))

Note: you may need to restart the kernel to use updated packages.
Setup successful.


In [5]:
import sys, os
print(sys.executable)
print(os.getcwd())

C:\Users\gmoores\AI-Tutorial-Codes-Included\venv\Scripts\python.exe
C:\Users\gmoores\AI-Tutorial-Codes-Included


In [6]:
# Block 3 (local model setup; no Gemini)
import os, json, time, ast, math
from dataclasses import dataclass, field
from typing import Dict, List, Callable, Any

# If networkx isn't installed, the graph still runs without drawing
try:
    import networkx as nx
except ImportError:
    nx = None

# Reuse local_chat if you already defined it earlier.
# If not, this fallback defines it here (expects your local server at 127.0.0.1:1234).
try:
    local_chat
except NameError:
    from openai import OpenAI
    BASE_URL = os.getenv("LLM_ENDPOINT", "http://127.0.0.1:1234/v1")
    API_KEY  = os.getenv("LLM_API_KEY", "sk-local")
    DEFAULT_MODEL = os.getenv("LLM_MODEL", "qwen/qwen2.5-vl-7b")
    client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

    def local_chat(prompt: str, *, system: str | None = None,
                   temperature: float = 0.2, max_tokens: int = 512,
                   model: str = DEFAULT_MODEL) -> str:
        msgs = []
        if system:
            msgs.append({"role": "system", "content": system})
        msgs.append({"role": "user", "content": prompt})
        resp = client.chat.completions.create(
            model=model,
            messages=msgs,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return resp.choices[0].message.content.strip()


In [7]:
# Block 4 (model factory + LLM call utility)
def make_model(model_name: str = "qwen/qwen2.5-vl-7b"):
    system_text = (
        "You are GraphAgent, a principled planner-executor. "
        "Prefer structured, concise outputs; use provided tools when asked."
    )
    # Return a callable that wraps our local_chat with a fixed system prompt & model
    def _call(prompt: str, temperature: float = 0.2):
        return local_chat(prompt, system=system_text, temperature=temperature, model=model_name)
    return _call

def call_llm(model, prompt: str, temperature=0.2) -> str:
    # model is the callable returned by make_model(...)
    return (model(prompt, temperature=temperature) or "").strip()


In [8]:
# Block 5 (math + toy RAG helpers)
def safe_eval_math(expr: str) -> str:
    node = ast.parse(expr, mode="eval")
    allowed = (ast.Expression, ast.BinOp, ast.UnaryOp, ast.Num, ast.Constant,
               ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.Mod,
               ast.USub, ast.UAdd, ast.FloorDiv, ast.AST)
    def check(n):
        if not isinstance(n, allowed):
            raise ValueError("Unsafe expression")
        for c in ast.iter_child_nodes(n):
            check(c)
    check(node)
    return str(eval(compile(node, "<math>", "eval"), {"__builtins__": {}}, {}))

DOCS = [
    "Solar panels convert sunlight to electricity; capacity factor ~20%.",
    "Wind turbines harvest kinetic energy; onshore capacity factor ~35%.",
    "RAG = retrieval-augmented generation joins search with prompting.",
    "LangGraph enables cyclic graphs of agents; good for tool orchestration.",
]
def search_docs(q: str, k: int = 3) -> List[str]:
    ql = q.lower()
    scored = sorted(DOCS, key=lambda d: -sum(w in d.lower() for w in ql.split()))
    return scored[:k]


In [9]:
# Block 6 (state + nodes + runner)
@dataclass
class State:
    task: str
    plan: str = ""
    scratch: List[str] = field(default_factory=list)
    evidence: List[str] = field(default_factory=list)
    result: str = ""
    step: int = 0
    done: bool = False

def node_plan(state: State, model) -> str:
    prompt = f"""Plan step-by-step to solve the user task.
Task: {state.task}
Return JSON: {{"subtasks": ["..."], "tools": {{"search": true/false, "math": true/false}}, "success_criteria": ["..."]}}"""
    js = call_llm(model, prompt)
    try:
        plan = json.loads(js[js.find("{"): js.rfind("}")+1])
    except Exception:
        plan = {"subtasks": ["Research", "Synthesize"], "tools": {"search": True, "math": False}, "success_criteria": ["clear answer"]}
    state.plan = json.dumps(plan, indent=2)
    state.scratch.append("PLAN:\n"+state.plan)
    return "route"

def node_route(state: State, model) -> str:
    prompt = f"""You are a router. Decide next node.
Context scratch:\n{chr(10).join(state.scratch[-5:])}
If math needed -> 'math', if research needed -> 'research', if ready -> 'write'.
Return one token from [research, math, write]. Task: {state.task}"""
    choice = call_llm(model, prompt).lower()
    if "math" in choice and any(ch.isdigit() for ch in state.task):
        return "math"
    if "research" in choice or not state.evidence:
        return "research"
    return "write"

def node_research(state: State, model) -> str:
    prompt = f"""Generate 3 focused search queries for:
Task: {state.task}
Return as JSON list of strings."""
    qjson = call_llm(model, prompt)
    try:
        queries = json.loads(qjson[qjson.find("["): qjson.rfind("]")+1])[:3]
    except Exception:
        queries = [state.task, "background "+state.task, "pros cons "+state.task]
    hits = []
    for q in queries:
        hits.extend(search_docs(q, k=2))
    # de-duplicate while preserving order
    state.evidence.extend(list(dict.fromkeys(hits)))
    state.scratch.append("EVIDENCE:\n- " + "\n- ".join(hits))
    return "route"

def node_math(state: State, model) -> str:
    prompt = "Extract a single arithmetic expression from this task:\n"+state.task
    expr = call_llm(model, prompt)
    expr = "".join(ch for ch in expr if ch in "0123456789+-*/().%^ ")
    try:
        val = safe_eval_math(expr)
        state.scratch.append(f"MATH: {expr} = {val}")
    except Exception as e:
        state.scratch.append(f"MATH-ERROR: {expr} ({e})")
    return "route"

def node_write(state: State, model) -> str:
    prompt = f"""Write the final answer.
Task: {state.task}
Use the evidence and any math results below, cite inline like [1],[2].
Evidence:\n{chr(10).join(f'[{i+1}] '+e for i,e in enumerate(state.evidence))}
Notes:\n{chr(10).join(state.scratch[-5:])}
Return a concise, structured answer."""
    draft = call_llm(model, prompt, temperature=0.3)
    state.result = draft
    state.scratch.append("DRAFT:\n"+draft)
    return "critic"

def node_critic(state: State, model) -> str:
    prompt = f"""Critique and improve the answer for factuality, missing steps, and clarity.
If fix needed, return improved answer. Else return 'OK'.
Answer:\n{state.result}\nCriteria:\n{state.plan}"""
    crit = call_llm(model, prompt)
    if crit.strip().upper() != "OK" and len(crit) > 30:
        state.result = crit.strip()
        state.scratch.append("REVISED")
    state.done = True
    return "end"

NODES: Dict[str, Callable[[State, Any], str]] = {
    "plan": node_plan, "route": node_route, "research": node_research,
    "math": node_math, "write": node_write, "critic": node_critic
}

def run_graph(task: str, model=None, model_name: str = "qwen/qwen2.5-vl-7b") -> State:
    model_callable = model or make_model(model_name)
    state = State(task=task)
    cur = "plan"
    max_steps = 12
    while not state.done and state.step < max_steps:
        state.step += 1
        nxt = NODES[cur](state, model_callable)
        if nxt == "end":
            break
        cur = nxt
    return state

def ascii_graph():
    return """
START -> plan -> route -> (research <-> route) & (math <-> route) -> write -> critic -> END
"""


In [10]:
# Block 7 (non-interactive runner)
import time

def run_once(task, model_name="qwen/qwen2.5-vl-7b"):
    print("TASK USED:\n", task, "\n")
    t0 = time.time()
    state = run_graph(task, model_name=model_name)
    dt = time.time() - t0
    print("\n=== GRAPH ===", ascii_graph())
    print(f"\n✅ Result in {dt:.2f}s:\n{state.result}\n")
    print("---- Evidence ----")
    print("\n".join(state.evidence))
    print("\n---- Scratch (last 5) ----")
    print("\n".join(state.scratch[-5:]))
    return state

task = (
    "Compare drought-tolerant gardening vs traditional gardening regarding each approach's "
    "aesthetics, maintenance requirements, and inputs (fertilizer, water, etc.) for DIY home "
    "landscapers redesigning Colorado front yards; compute 5*7"
)
state = run_once(task)


📝 Enter your task:  📝 Enter your task:  Compare drought-tolerant gardening vs traditional gardening regarding each approaches aesthetics, maintenance requirements, and amount of inputs (fertilizer, water, etc.) for DIY home landscapers looking to redesign their Colorado front yards ; compute 5*7



=== GRAPH === 
START -> plan -> route -> (research <-> route) & (math <-> route) -> write -> critic -> END


✅ Result in 10.30s:


---- Evidence ----
Solar panels convert sunlight to electricity; capacity factor ~20%.
Wind turbines harvest kinetic energy; onshore capacity factor ~35%.
RAG = retrieval-augmented generation joins search with prompting.
LangGraph enables cyclic graphs of agents; good for tool orchestration.
Solar panels convert sunlight to electricity; capacity factor ~20%.
Wind turbines harvest kinetic energy; onshore capacity factor ~35%.
RAG = retrieval-augmented generation joins search with prompting.
Solar panels convert sunlight to electricity; capacity factor ~20%.
Wind turbines harvest kinetic energy; onshore capacity factor ~35%.
RAG = retrieval-augmented generation joins search with prompting.
Wind turbines harvest kinetic energy; onshore capacity factor ~35%.
RAG = retrieval-augmented generation joins search with prompting.
Solar panels convert sunlight to elec